In [2]:
# play audio in ipython notebook
try:
    import winsound
except ImportError:
    !pip install winsound
    import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [3]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
from deepface import DeepFace
import warnings
warnings.filterwarnings("ignore")

tt = time.time
st = tt()

Using TensorFlow backend.
C:\Users\YASH\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\YASH\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\YASH\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\YASH\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

time: 14.7 s


In [4]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

time: 11.5 ms


In [5]:
base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

known_faces_dir = os.path.join(data_dir, 'known_faces')
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('\\')[-1])] for w in images_dir_list]

# known_face_images_nested_list = []
# for image_dir in images_dir_list:
#     known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
#     known_face_images_nested_list.append([face_recognition.load_image_file(w) for w in known_images_path_list])

# save_as_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"), known_face_names)
# save_as_pickle(os.path.join(pkl_files_dir, "known_face_images_nested_list.pkl"), known_face_encodings)
# save_as_pickle(os.path.join(pkl_files_dir, "known_face_seprate_encodings.pkl"), known_face_seprate_encodings)

# winsound.Beep(freq, duration)

time: 9.97 ms


In [7]:
tracebacks_dir = os.path.join(data_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

pkl_files_dir = os.path.join(data_dir, "pkl_files")
nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

known_face_names = load_from_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"))
known_face_encodings = load_from_pickle(os.path.join(pkl_files_dir, "known_face_encodings.pkl"))

# backends = ['opencv', 'ssd', 'dlib', 'mtcnn']
backends = ['opencv', 'ssd', 'dlib']

# start after labeled faces
vedio_start_idx = 2
# treshold for face distance
max_pred_cnt = 20

num_channels = 5
num_playlists = 5
num_vedios = 5
    
# num_channels = 5
# num_playlists = 5
# num_vedios = 5

winner_backend=[]
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)

    results_chanel_dir = os.path.join(results_dir, nptel_channel_name)
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
    paylist_name_list = [x.split('\\')[-1] for x in paylist_path_list] 
    
    vedio_ext = '.mp4'
    
    namef=0
    namenf=0
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
    for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
        print(f"\n\nplaylist: {playlistn}")
        if playlistn not in playlist_2_prof_name_dict.keys():
            print(f"{playlistn} not in playlist_2_prof_name_dict")
            continue
        
        true_name = playlist_2_prof_name_dict[playlistn]
        if true_name is None:
            print(f"no name for {playlistn}")
            continue
            
        if true_name not in folder_name_2_prof_name_dict.values():
            print(f"no face for {true_name}")
            continue
        
        results_playlist_dir = os.path.join(results_chanel_dir, playlistn)
        
        vedio_path_list = get_extension_files(playlist, ext='.mp4')
        vedio_name_list = [x.split('\\')[-1] for x in vedio_path_list] 
        
        for vedio_path in vedio_path_list[vedio_start_idx:vedio_start_idx+num_vedios]:
            vname = vedio_path.split('\\')[-1]

            video_capture = cv2.VideoCapture(vedio_path)

            vst = 0*60
            vet = 5*60
            num_frames_per_sec = 1
            pred_cnt = 0
            names_list = []

            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                 # Grab a single frame of video
                ret, frame = video_capture.read()

                # process
                if i%int(fps/num_frames_per_sec)==0:
                    
                    atleastone = False
                    for backend in backends:
                        try:
                            # TODO: PLS find a workaround this is too stupid
                            cv2.imwrite("img.jpg", frame)
                            detected_face = DeepFace.detectFace("img.jpg", detector_backend = backend)
                            names_list.append(backend)
                            atleastone = True
                        except ValueError:
                            pass
                    
                    if atleastone:
                        pred_cnt+=1
                        if pred_cnt>=max_pred_cnt:
                            break
                    
                # Hit 'q' on the keyboard to quit!
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release handle to the webcam
            video_capture.release()
            cv2.destroyAllWindows()
            
            print(f"\nvedio: {vname}")
            print(f"\ntime: {time.time()-st}")
            print("predicted face counts:")
            unique, counts = np.unique(np.array(names_list), return_counts=True)
            print(np.asarray((unique, counts)).T)
            
            max_backend = max(names_list, key=names_list.count)
            winner_backend.append(max_backend)
            max_cnt = len([w for w in names_list if w ==max_backend])
            print(f"max predictions: {max_backend} number of predictions: {max_cnt}")

print("\n\nwinner backends in all vedios:")
unique, counts = np.unique(np.array(winner_backend), return_counts=True)
print(np.asarray((unique, counts)).T)

winsound.Beep(freq, duration)




for nptel_channel_name Aerospace Engineering


playlist: Foundation of Scientific Computing
no face for prof. t.k.sengupta


playlist: Instability and Transition of Fluid Flows
Instability and Transition of Fluid Flows not in playlist_2_prof_name_dict


playlist: Introduction to Aerodynamics

vedio: VhybIE82B6M.mp4

time: 120.91571164131165
predicted face counts:
[['dlib' '19']
 ['opencv' '18']
 ['ssd' '20']]
max predictions: ssd number of predictions: 20

vedio: _9qwbichO1s.mp4

time: 124.19435477256775
predicted face counts:
[['dlib' '20']
 ['opencv' '18']
 ['ssd' '20']]
max predictions: ssd number of predictions: 20


playlist: Jet and Rocket Propulsion

vedio: UcDNM18TkiU.mp4

time: 157.7942612171173
predicted face counts:
[['dlib' '20']
 ['opencv' '20']
 ['ssd' '17']]
max predictions: opencv number of predictions: 20

vedio: YrkbOeF7wFw.mp4

time: 245.50769758224487
predicted face counts:
[['dlib' '18']
 ['opencv' '17']
 ['ssd' '15']]
max predictions: dlib number of predictions


vedio: T6xDcGWOEro.mp4

time: 44.08434534072876
predicted face counts:
[['dlib' '20']
 ['opencv' '20']
 ['ssd' '4']]
max predictions: opencv number of predictions: 20



for nptel_channel_name Computer Science and Engineering


playlist: Artificial Intelligence Search Methods for problem Solving

vedio: ws89tmbpt8w.mp4

time: 38.812392234802246
predicted face counts:
[['dlib' '20']
 ['opencv' '20']
 ['ssd' '8']]
max predictions: opencv number of predictions: 20

vedio: X9PhasOXlT8.mp4

time: 41.91766285896301
predicted face counts:
[['dlib' '20']
 ['opencv' '18']
 ['ssd' '18']]
max predictions: dlib number of predictions: 20

vedio: _MhgsoPHvYo.mp4

time: 43.08152675628662
predicted face counts:
[['dlib' '18']
 ['opencv' '18']
 ['ssd' '20']]
max predictions: ssd number of predictions: 20


playlist: Cloud Computing

vedio: SqG-b5E9vHs.mp4

time: 64.57273459434509
predicted face counts:
[['dlib' '16']
 ['opencv' '20']
 ['ssd' '14']]
max predictions: opencv number of predictions: 20


p